Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) 

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
train_subset = 10000 

graph = tf.Graph()
with graph.as_default():
    
    # put the data in tensor format
    tf_train = tf.constant(train_dataset[:train_subset])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid = tf.constant(valid_dataset)
    tf_test = tf.constant(test_dataset)
    
    # set variables 
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels], seed=42))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train, weights) + biases 
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) 
                         + .009*tf.nn.l2_loss(weights))
              
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test, weights) + biases)

In [6]:
num_steps = 801

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 ==0):
            print('Loss at step {0}: {1}'.format(step, l))
            print('Training accuracy: {0}'.format(accuracy(predictions,
                                                           train_labels[:train_subset, :])))
            print("Validation Accuracy: {0}".format(accuracy(valid_prediction.eval(),
                                                               valid_labels)))
    print("Test Accuracy: {0}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Loss at step 0: 44.2741584778
Training accuracy: 9.61
Validation Accuracy: 13.18
Loss at step 100: 11.7622814178
Training accuracy: 73.1
Validation Accuracy: 72.19
Loss at step 200: 4.85589981079
Training accuracy: 78.13
Validation Accuracy: 76.0
Loss at step 300: 2.26183199883
Training accuracy: 81.67
Validation Accuracy: 78.75
Loss at step 400: 1.28529012203
Training accuracy: 83.37
Validation Accuracy: 80.61
Loss at step 500: 0.913721859455
Training accuracy: 84.21
Validation Accuracy: 81.43
Loss at step 600: 0.770832359791
Training accuracy: 84.35
Validation Accuracy: 81.69
Loss at step 700: 0.715430915356
Training accuracy: 84.41
Validation Accuracy: 81.88
Loss at step 800: 0.693813085556
Training accuracy: 84.45
Validation Accuracy: 81.85
Test Accuracy: 88.91


By adding a value of .004*penalty we see that we can increase the score a full point. 

I'm going to try the same on the ReLu configuration. 

In [7]:
train_subset = 10000 
hidden_nodes = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    # put the data in tensor format
    tf_train = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid = tf.constant(valid_dataset)
    tf_test = tf.constant(test_dataset)
    
    # set variables 
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_nodes], seed=42))
    biases = tf.Variable(tf.zeros([hidden_nodes]))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels], seed=42))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train, weights) + biases 
    
    hidden_layer = tf.nn.relu(logits)
    
    logits2 = tf.matmul(hidden_layer, weights2) + biases2 
    
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2)) + .004*tf.nn.l2_loss(weights)
           )
             
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits2)
    
    # validation prediciton
    logits = tf.matmul(tf_valid, weights) + biases
    hidden_layer = tf.nn.relu(logits)
    logits2 = tf.matmul(hidden_layer, weights2) + biases2
    valid_prediction = tf.nn.softmax(logits2)

    # Predictions for test data.
    logits = tf.matmul(tf_test, weights) + biases
    hidden_layer = tf.nn.relu(logits)
    logits2 = tf.matmul(hidden_layer, weights2) + biases2

    test_prediction = tf.nn.softmax(logits2)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:  
  tf.initialize_all_variables().run()  
  print("Initialized")  
  for step in range(num_steps):  
    # create offset
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)  
    
    # Generate a minibatch.  
    batch_data = train_dataset[offset:(offset + batch_size), :]  
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # below we do stochasic training by feeding randomized batches of the data over each epoch,
    # mapping them to the placeholder tf_train_dataset  
    feed_dict = {tf_train : batch_data, tf_train_labels : batch_labels}  
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)  
    
    if (step % 500 == 0):  
      print("Minibatch loss at step %d: %f" % (step, l))  
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))  
      print("Validation accuracy: %.1f%%" % accuracy(  
        valid_prediction.eval(), valid_labels))  
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 1568.608643
Minibatch accuracy: 10.2%
Validation accuracy: 24.6%
Minibatch loss at step 500: 169.189529
Minibatch accuracy: 82.0%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 22.956333
Minibatch accuracy: 81.2%
Validation accuracy: 84.0%
Minibatch loss at step 1500: 3.661397
Minibatch accuracy: 82.8%
Validation accuracy: 85.1%
Minibatch loss at step 2000: 1.048045
Minibatch accuracy: 82.8%
Validation accuracy: 84.4%
Minibatch loss at step 2500: 0.474478
Minibatch accuracy: 88.3%
Validation accuracy: 85.3%
Minibatch loss at step 3000: 0.560342
Minibatch accuracy: 85.2%
Validation accuracy: 86.0%
Test accuracy: 92.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [38]:
train_subset = 10000 
hidden_nodes = 1024
batch_size = 8

graph = tf.Graph()
with graph.as_default():
    
    # put the data in tensor format
    tf_train = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid = tf.constant(valid_dataset)
    tf_test = tf.constant(test_dataset)
    
    # set variables 
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_nodes], seed=42))
    biases = tf.Variable(tf.zeros([hidden_nodes]))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels], seed=42))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train, weights) + biases 
    
    hidden_layer = tf.nn.relu(logits)
    
    logits2 = tf.matmul(hidden_layer, weights2) + biases2 
    
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2) + .000000004*tf.nn.l2_loss(weights) )
           )
             
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits2)
    
    # validation prediciton
    logits = tf.matmul(tf_valid, weights) + biases
    hidden_layer = tf.nn.relu(logits)
    logits2 = tf.matmul(hidden_layer, weights2) + biases2
    valid_prediction = tf.nn.softmax(logits2)

    # Predictions for test data.
    logits = tf.matmul(tf_test, weights) + biases
    hidden_layer = tf.nn.relu(logits)
    logits2 = tf.matmul(hidden_layer, weights2) + biases2

    test_prediction = tf.nn.softmax(logits2)

In [39]:
num_steps = 3001

with tf.Session(graph=graph) as session:  
  tf.initialize_all_variables().run()  
  print("Initialized")  
  for step in range(num_steps):  
    # create offset
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)  
    
    # Generate a minibatch.  
    batch_data = train_dataset[offset:(offset + batch_size), :]  
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # below we do stochasic training by feeding randomized batches of the data over each epoch,
    # mapping them to the placeholder tf_train_dataset  
    feed_dict = {tf_train : batch_data, tf_train_labels : batch_labels}  
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)  
    
    if (step % 500 == 0):  
      print("Minibatch loss at step %d: %f" % (step, l))  
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))  
      print("Validation accuracy: %.1f%%" % accuracy(  
        valid_prediction.eval(), valid_labels))  
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 285.200806
Minibatch accuracy: 0.0%
Validation accuracy: 13.4%
Minibatch loss at step 500: 13463486464.000000
Minibatch accuracy: 25.0%
Validation accuracy: 41.0%
Minibatch loss at step 1000: 113810551603200.000000
Minibatch accuracy: 25.0%
Validation accuracy: 38.7%
Minibatch loss at step 1500: 311071630707130368.000000
Minibatch accuracy: 25.0%
Validation accuracy: 41.9%
Minibatch loss at step 2000: 5848557434838188032000.000000
Minibatch accuracy: 37.5%
Validation accuracy: 33.3%
Minibatch loss at step 2500: 219520718589209501941891072.000000
Minibatch accuracy: 12.5%
Validation accuracy: 36.4%
Minibatch loss at step 3000: 587406326534099236155805401088.000000
Minibatch accuracy: 87.5%
Validation accuracy: 42.9%
Test accuracy: 47.5%


As we can see, using a small amount of mini batches causes extreme overfitting during training and leads to 
poor test scores. 

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [53]:
train_subset = 100000 
hidden_nodes = 1024
batch_size = 15

graph = tf.Graph()
with graph.as_default():
    
    # put the data in tensor format
    tf_train = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid = tf.constant(valid_dataset)
    tf_test = tf.constant(test_dataset)
    
    # set variables 
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_nodes], seed=42))
    biases = tf.Variable(tf.zeros([hidden_nodes]))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels], seed=42))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train, weights) + biases 
    
    hidden_layer = tf.nn.relu(logits)
    
    keep_prob = tf.placeholder(tf.float32)
    drop_layer = tf.nn.dropout(hidden_layer, keep_prob)
    
    logits2 = tf.matmul(drop_layer, weights2) + biases2 
    
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2) + .000000004*tf.nn.l2_loss(weights) )
           )
             
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits2)
    
    # validation prediciton
    logits = tf.matmul(tf_valid, weights) + biases
    hidden_layer = tf.nn.relu(logits)
    logits2 = tf.matmul(hidden_layer, weights2) + biases2
    valid_prediction = tf.nn.softmax(logits2)

    # Predictions for test data.
    logits = tf.matmul(tf_test, weights) + biases
    hidden_layer = tf.nn.relu(logits)
    logits2 = tf.matmul(hidden_layer, weights2) + biases2

    test_prediction = tf.nn.softmax(logits2)

In [57]:
num_steps = 3001

with tf.Session(graph=graph) as session:  
  tf.initialize_all_variables().run()  
  print("Initialized")  
  for step in range(num_steps):  
    # create offset
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)  
    
    # Generate a minibatch.  
    batch_data = train_dataset[offset:(offset + batch_size), :]  
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # below we do stochasic training by feeding randomized batches of the data over each epoch,
    # mapping them to the placeholder tf_train_dataset  
    feed_dict = {tf_train : batch_data, tf_train_labels : batch_labels, keep_prob:.8}  
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)  
    
    if (step % 500 == 0):  
      print("Minibatch loss at step %d: %f" % (step, l))  
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))  
      print("Validation accuracy: %.1f%%" % accuracy(  
        valid_prediction.eval(), valid_labels))  
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 417.025635
Minibatch accuracy: 6.7%
Validation accuracy: 11.2%
Minibatch loss at step 500: 80897.656250
Minibatch accuracy: 53.3%
Validation accuracy: 54.5%
Minibatch loss at step 1000: 416017.812500
Minibatch accuracy: 60.0%
Validation accuracy: 52.0%
Minibatch loss at step 1500: 4424503.000000
Minibatch accuracy: 73.3%
Validation accuracy: 58.3%
Minibatch loss at step 2000: 9065554.000000
Minibatch accuracy: 46.7%
Validation accuracy: 56.7%
Minibatch loss at step 2500: 3777035.750000
Minibatch accuracy: 60.0%
Validation accuracy: 60.5%
Minibatch loss at step 3000: 900589952.000000
Minibatch accuracy: 46.7%
Validation accuracy: 55.8%
Test accuracy: 61.2%


As we cam see, if we have the probability of keeping each node at .80 it prevents the model from overfitting on the mini batches. 

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [ ]:
# to do: 

# return and create complex model 